# Netflix Data Cleaning & Feature Engineering 

This notebook provides comprehensive data cleaning, validation, and feature engineering for the Netflix dataset analysis pipeline.

##  Processing Steps
1. **Data Loading & Initial Assessment** - Load raw data and assess quality
2. **Data Cleaning & Standardization** - Handle missing values, duplicates, and format issues
3. **Temporal Feature Engineering** - Create date-based and temporal features
4. **Geographic Feature Engineering** - Process country and regional data
5. **Content Feature Engineering** - Create content-specific features
6. **Text Feature Engineering** - Process descriptions and text data
7. **Derived Analytics Features** - Create business intelligence features
8. **Data Quality Validation** - Comprehensive quality checks
9. **Final Export** - Save cleaned dataset for downstream analysis

##  Output Features
- **Temporal**: `date_added_year`, `content_age_when_added`, `is_recent_content`, `decade_released`
- **Geographic**: `primary_country`, `continent`, `country_count`, `is_international`
- **Content**: `duration_minutes`, `primary_genre`, `genre_count`, `is_adult_content`, `is_family_friendly`
- **Analytics**: `is_classic`, `is_short_content`, `is_long_content`, `years_since_release`
- **Text**: `description_length`, `description_word_count`


In [2]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import warnings
from datetime import datetime, timedelta
import re
from typing import Dict, List, Optional
import logging
import random

# Set random seed for reproducible description generation
np.random.seed(42)
random.seed(42)

# Add src directory to path
sys.path.append('../src')
from utils import (connect_db, get_engine, parse_duration, clean_countries, 
                   clean_genres, get_continent_mapping, print_data_summary, 
                   setup_plotting_style)

# Configure settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
warnings.filterwarnings('ignore')
setup_plotting_style()

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Create necessary directories
os.makedirs('../data/processed', exist_ok=True)
os.makedirs('../data/interim', exist_ok=True)
os.makedirs('../reports/data_quality', exist_ok=True)

print("Netflix Data Cleaning & Feature Engineering Pipeline")
print(f"Processing Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


Netflix Data Cleaning & Feature Engineering Pipeline
Processing Started: 2025-07-01 08:47:40


---
## 1. Data Loading & Initial Assessment 


In [4]:
print("1. DATA LOADING & INITIAL ASSESSMENT ")


# Load raw Netflix data
print("\n1.1 Loading Raw Dataset")


try:
    # Try loading from different possible locations
    possible_paths = [
        '../data/raw/netflix_raw.csv',
        '../netflix1.csv',
        '../data/netflix1.csv'
    ]
    
    df_raw = None
    for path in possible_paths:
        try:
            df_raw = pd.read_csv(path)
            data_source = path
            print(f"Successfully loaded data from: {path}")
            break
        except FileNotFoundError:
            continue
    
    if df_raw is None:
        raise FileNotFoundError("Could not find Netflix dataset in expected locations")
        
    print(f"Raw dataset shape: {df_raw.shape}")
    
except Exception as e:
    print(f" Error loading data: {e}")
    print(" Expected file locations:")
    for path in possible_paths:
        print(f"   - {path}")
    raise

# Initial data inspection
print(f"\n1.2 Initial Data Quality Assessment")

print(f"• Dataset dimensions: {df_raw.shape[0]:,} rows × {df_raw.shape[1]} columns")
print(f"• Memory usage: {df_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Column information
print(f"\n• Column Overview:")
for i, (col, dtype) in enumerate(zip(df_raw.columns, df_raw.dtypes), 1):
    null_count = df_raw[col].isnull().sum()
    null_pct = (null_count / len(df_raw)) * 100
    print(f"  {i:2d}. {col:20s} | {str(dtype):12s} | {null_count:5d} nulls ({null_pct:5.1f}%)")

# Data sample
print(f"\n• Data Sample (first 3 rows):")
display_columns = ['title', 'type', 'country', 'release_year', 'rating', 'duration']
print(df_raw[display_columns].head(3).to_string(index=False))

# Save initial assessment
assessment_summary = pd.DataFrame({
    'Column': df_raw.columns,
    'Data_Type': df_raw.dtypes.astype(str),
    'Non_Null_Count': df_raw.count(),
    'Null_Count': df_raw.isnull().sum(),
    'Null_Percentage': (df_raw.isnull().sum() / len(df_raw) * 100).round(2),
    'Unique_Values': [df_raw[col].nunique() for col in df_raw.columns]
})

assessment_summary.to_csv('../reports/data_quality/01_initial_assessment.csv', index=False)
print(f"\n Initial assessment saved to: reports/data_quality/01_initial_assessment.csv")


1. DATA LOADING & INITIAL ASSESSMENT 

1.1 Loading Raw Dataset
Successfully loaded data from: ../data/raw/netflix_raw.csv
Raw dataset shape: (8790, 10)

1.2 Initial Data Quality Assessment
• Dataset dimensions: 8,790 rows × 10 columns
• Memory usage: 4.66 MB

• Column Overview:
   1. show_id              | object       |     0 nulls (  0.0%)
   2. type                 | object       |     0 nulls (  0.0%)
   3. title                | object       |     0 nulls (  0.0%)
   4. director             | object       |     0 nulls (  0.0%)
   5. country              | object       |     0 nulls (  0.0%)
   6. date_added           | object       |     0 nulls (  0.0%)
   7. release_year         | int64        |     0 nulls (  0.0%)
   8. rating               | object       |     0 nulls (  0.0%)
   9. duration             | object       |     0 nulls (  0.0%)
  10. listed_in            | object       |     0 nulls (  0.0%)

• Data Sample (first 3 rows):
               title    type       count

---
## 2. Data Cleaning & Standardization 


In [6]:
print("2. DATA CLEANING & STANDARDIZATION")


# Create working copy
df_clean = df_raw.copy()
print(f"Created working copy with {len(df_clean):,} records")

# 2.1 Handle Missing Values
print(f"\n2.1 Missing Value Treatment")

# Identify missing value patterns
missing_summary = df_clean.isnull().sum().sort_values(ascending=False)
missing_summary = missing_summary[missing_summary > 0]

if len(missing_summary) > 0:
    print("• Missing value counts:")
    for col, count in missing_summary.items():
        pct = (count / len(df_clean)) * 100
        print(f"  - {col}: {count:,} ({pct:.1f}%)")
    
    # Handle missing values based on column type and business logic
    
    # Director: Fill with 'Unknown Director' and ensure string type
    if 'director' in df_clean.columns:
        before_null = df_clean['director'].isnull().sum()
        df_clean['director'] = df_clean['director'].astype(str).replace(['nan', 'None'], 'Unknown Director')
        df_clean['director'] = df_clean['director'].fillna('Unknown Director')
        print(f"Filled {before_null:,} missing director values with 'Unknown Director'")
    
    # Cast: Fill with 'Unknown Cast' and ensure string type
    if 'cast' in df_clean.columns:
        before_null = df_clean['cast'].isnull().sum()
        df_clean['cast'] = df_clean['cast'].astype(str).replace(['nan', 'None'], 'Unknown Cast')
        df_clean['cast'] = df_clean['cast'].fillna('Unknown Cast')
        print(f"Filled {before_null:,} missing cast values with 'Unknown Cast'")
    
    # Country: Fill with 'Unknown Country'
    if 'country' in df_clean.columns:
        before_null = df_clean['country'].isnull().sum()
        df_clean['country'] = df_clean['country'].fillna('Unknown Country')
        print(f"Filled {before_null:,} missing country values with 'Unknown Country'")
    
    # Date added: Remove rows with missing date_added (critical for analysis)
    if 'date_added' in df_clean.columns:
        before_null = df_clean['date_added'].isnull().sum()
        if before_null > 0:
            df_clean = df_clean.dropna(subset=['date_added'])
            print(f"Removed {before_null:,} rows with missing date_added")
    
    # Rating: Fill with most common rating
    if 'rating' in df_clean.columns:
        before_null = df_clean['rating'].isnull().sum()
        if before_null > 0:
            most_common_rating = df_clean['rating'].mode().iloc[0]
            df_clean['rating'] = df_clean['rating'].fillna(most_common_rating)
            print(f"Filled {before_null:,} missing rating values with '{most_common_rating}'")
    
    # Description: Fill with generic description and ensure string type
    if 'description' in df_clean.columns:
        before_null = df_clean['description'].isnull().sum()
        df_clean['description'] = df_clean['description'].astype(str).replace(['nan', 'None'], 'No description available')
        df_clean['description'] = df_clean['description'].fillna('No description available')
        print(f"Filled {before_null:,} missing description values")
        
else:
    print("• No missing values found in dataset")

# 2.2 Remove Duplicates
print(f"\n2.2 Duplicate Detection & Removal")

# Check for exact duplicates
exact_duplicates = df_clean.duplicated().sum()
if exact_duplicates > 0:
    df_clean = df_clean.drop_duplicates()
    print(f"Removed {exact_duplicates:,} exact duplicate rows")
else:
    print("No exact duplicates found")

# Check for title-based duplicates (same title, type, release year)
if all(col in df_clean.columns for col in ['title', 'type', 'release_year']):
    title_duplicates = df_clean.duplicated(subset=['title', 'type', 'release_year']).sum()
    if title_duplicates > 0:
        print(f"Found {title_duplicates:,} potential content duplicates (same title, type, year)")
        # Keep first occurrence
        df_clean = df_clean.drop_duplicates(subset=['title', 'type', 'release_year'], keep='first')
        print(f"Kept first occurrence of each duplicate")
    else:
        print("No content duplicates found")

print(f"Final dataset after cleaning: {len(df_clean):,} records")

# 2.3 Data Type Standardization
print(f"\n2.3 Data Type Standardization")

# Standardize column names (lowercase, replace spaces with underscores)
original_columns = df_clean.columns.tolist()
df_clean.columns = df_clean.columns.str.lower().str.replace(' ', '_')
renamed_columns = df_clean.columns.tolist()

if original_columns != renamed_columns:
    print("Standardized column names to lowercase with underscores")
    
# Ensure proper data types
if 'release_year' in df_clean.columns:
    df_clean['release_year'] = pd.to_numeric(df_clean['release_year'], errors='coerce')
    print("Converted release_year to numeric")

# Clean text columns (remove extra whitespace)
text_columns = df_clean.select_dtypes(include=['object']).columns
for col in text_columns:
    df_clean[col] = df_clean[col].astype(str).str.strip()
print(f"Cleaned whitespace from {len(text_columns)} text columns")

print(f"\n Data cleaning completed successfully!")


2. DATA CLEANING & STANDARDIZATION
Created working copy with 8,790 records

2.1 Missing Value Treatment
• No missing values found in dataset

2.2 Duplicate Detection & Removal
No exact duplicates found
Found 3 potential content duplicates (same title, type, year)
Kept first occurrence of each duplicate
Final dataset after cleaning: 8,787 records

2.3 Data Type Standardization
Converted release_year to numeric
Cleaned whitespace from 9 text columns

 Data cleaning completed successfully!


---
## 3. Temporal Feature Engineering


In [7]:
print("3. TEMPORAL FEATURE ENGINEERING")


# 3.1 Date Processing
print(f"\n3.1 Date Column Processing")

if 'date_added' in df_clean.columns:
    # Convert to datetime
    df_clean['date_added'] = pd.to_datetime(df_clean['date_added'], errors='coerce')
    
    # Extract date components
    df_clean['date_added_year'] = df_clean['date_added'].dt.year
    df_clean['date_added_month'] = df_clean['date_added'].dt.month
    df_clean['date_added_day'] = df_clean['date_added'].dt.day
    df_clean['date_added_weekday'] = df_clean['date_added'].dt.dayofweek  # 0=Monday
    df_clean['date_added_quarter'] = df_clean['date_added'].dt.quarter
    
    print(f"Extracted year, month, day, weekday, and quarter from date_added")
    print(f"Date range: {df_clean['date_added'].min().strftime('%Y-%m-%d')} to {df_clean['date_added'].max().strftime('%Y-%m-%d')}")
else:
    print("No date_added column found")

# 3.2 Content Age Features
print(f"\n3.2 Content Age Feature Creation")

if 'release_year' in df_clean.columns and 'date_added_year' in df_clean.columns:
    # Content age when added to Netflix
    df_clean['content_age_when_added'] = df_clean['date_added_year'] - df_clean['release_year']
    
    # Recent content flag (added within 2 years of release)
    df_clean['is_recent_content'] = df_clean['content_age_when_added'] <= 2
    
    # Content age categories
    df_clean['content_age_category'] = pd.cut(
        df_clean['content_age_when_added'], 
        bins=[-np.inf, 0, 2, 5, 10, 20, np.inf],
        labels=['Future_Release', 'Very_Recent', 'Recent', 'Moderate', 'Old', 'Classic']
    )
    
    print(f"Created content_age_when_added (range: {df_clean['content_age_when_added'].min():.0f} to {df_clean['content_age_when_added'].max():.0f} years)")
    print(f"Created is_recent_content flag ({df_clean['is_recent_content'].sum():,} recent titles)")
    print(f"Created content_age_category with 6 levels")
    
    # Content freshness score (inverse of age, normalized)
    max_age = df_clean['content_age_when_added'].max()
    df_clean['content_freshness_score'] = 1 - (df_clean['content_age_when_added'] / max_age)
    df_clean['content_freshness_score'] = df_clean['content_freshness_score'].clip(0, 1)
    print(f"Created content_freshness_score (0-1 scale)")

# 3.3 Release Era Features
print(f"\n3.3 Release Era Feature Creation")

if 'release_year' in df_clean.columns:
    # Decade classification
    df_clean['decade_released'] = (df_clean['release_year'] // 10) * 10
    
    # Era classification
    def classify_era(year):
        if year < 1970:
            return 'Classic_Era'
        elif year < 1990:
            return 'Golden_Age'
        elif year < 2000:
            return 'Modern_Era'
        elif year < 2010:
            return 'Digital_Era'
        else:
            return 'Streaming_Era'
    
    df_clean['release_era'] = df_clean['release_year'].apply(classify_era)
    
    # Years since release (from current year)
    current_year = datetime.now().year
    df_clean['years_since_release'] = current_year - df_clean['release_year']
    
    # Classic content flag (>20 years old)
    df_clean['is_classic'] = df_clean['years_since_release'] > 20
    
    print(f"Created decade_released ({df_clean['decade_released'].nunique()} unique decades)")
    print(f"Created release_era categories: {df_clean['release_era'].value_counts().to_dict()}")
    print(f"Created years_since_release and is_classic flag ({df_clean['is_classic'].sum():,} classic titles)")

# 3.4 Temporal Trends
print(f"\n3.4 Temporal Trend Features")

if 'date_added_year' in df_clean.columns:
    # Netflix growth phase classification
    def classify_netflix_phase(year):
        if year < 2013:
            return 'Early_Phase'
        elif year < 2016:
            return 'Growth_Phase'
        elif year < 2019:
            return 'Expansion_Phase'
        else:
            return 'Global_Phase'
    
    df_clean['netflix_phase'] = df_clean['date_added_year'].apply(classify_netflix_phase)
    
    # Month seasonality features
    month_names = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
                   7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
    df_clean['month_name'] = df_clean['date_added_month'].map(month_names)
    
    # Season classification
    def classify_season(month):
        if month in [12, 1, 2]:
            return 'Winter'
        elif month in [3, 4, 5]:
            return 'Spring'
        elif month in [6, 7, 8]:
            return 'Summer'
        else:
            return 'Fall'
    
    df_clean['season_added'] = df_clean['date_added_month'].apply(classify_season)
    
    print(f"Created netflix_phase categories: {df_clean['netflix_phase'].value_counts().to_dict()}")
    print(f"Created month_name and season_added features")

temporal_features = ['date_added_year', 'date_added_month', 'content_age_when_added', 
                    'is_recent_content', 'decade_released', 'years_since_release', 'is_classic']
print(f"\n Created {len(temporal_features)} temporal features successfully!")


3. TEMPORAL FEATURE ENGINEERING

3.1 Date Column Processing
Extracted year, month, day, weekday, and quarter from date_added
Date range: 2008-01-01 to 2021-09-25

3.2 Content Age Feature Creation
Created content_age_when_added (range: -3 to 93 years)
Created is_recent_content flag (5,547 recent titles)
Created content_age_category with 6 levels
Created content_freshness_score (0-1 scale)

3.3 Release Era Feature Creation
Created decade_released (10 unique decades)
Created release_era categories: {'Streaming_Era': 7455, 'Digital_Era': 807, 'Modern_Era': 274, 'Golden_Age': 199, 'Classic_Era': 52}
Created years_since_release and is_classic flag (781 classic titles)

3.4 Temporal Trend Features
Created netflix_phase categories: {'Global_Phase': 5391, 'Expansion_Phase': 3258, 'Growth_Phase': 117, 'Early_Phase': 21}
Created month_name and season_added features

 Created 7 temporal features successfully!


---
## 4. Geographic Feature Engineering 


In [8]:
print("4. GEOGRAPHIC FEATURE ENGINEERING")


# 4.1 Country Processing
print(f"\n4.1 Country Data Processing")

if 'country' in df_clean.columns:
    # Get primary country (first country listed)
    df_clean['primary_country'] = df_clean['country'].str.split(',').str[0].str.strip()
    
    # Count number of countries per title
    df_clean['country_count'] = df_clean['country'].str.split(',').str.len()
    df_clean['country_count'] = df_clean['country_count'].fillna(1)  # Single country if no comma
    
    # International production flag
    df_clean['is_international'] = df_clean['country_count'] > 1
    
    # Clean up country names (handle common variations)
    country_mapping = {
        'United States': 'United States',
        'United Kingdom': 'United Kingdom', 
        'India': 'India',
        'South Korea': 'South Korea',
        'Japan': 'Japan',
        'Canada': 'Canada',
        'France': 'France',
        'Germany': 'Germany',
        'Spain': 'Spain',
        'Italy': 'Italy',
        'Australia': 'Australia',
        'Brazil': 'Brazil',
        'Mexico': 'Mexico',
        'Netherlands': 'Netherlands',
        'Turkey': 'Turkey'
    }
    
    # Apply country name standardization
    df_clean['primary_country_clean'] = df_clean['primary_country'].replace(country_mapping)
    
    print(f"  Extracted primary_country ({df_clean['primary_country'].nunique()} unique countries)")
    print(f"  Created country_count (range: {df_clean['country_count'].min():.0f} to {df_clean['country_count'].max():.0f})")
    print(f"  Created is_international flag ({df_clean['is_international'].sum():,} international productions)")

# 4.2 Continent Mapping
print(f"\n4.2 Continent Classification")

# Get continent mapping from utils
continent_mapping = get_continent_mapping()

# Map countries to continents
df_clean['continent'] = df_clean['primary_country'].map(continent_mapping)

# Fill unknown continents
df_clean['continent'] = df_clean['continent'].fillna('Unknown')

continent_counts = df_clean['continent'].value_counts()
print(f"  Mapped countries to continents:")
for continent, count in continent_counts.items():
    pct = (count / len(df_clean)) * 100
    print(f"    - {continent}: {count:,} titles ({pct:.1f}%)")

# 4.3 Regional Classifications
print(f"\n4.3 Regional Classification Features")

# Major content markets
major_markets = ['United States', 'India', 'United Kingdom', 'South Korea', 'Japan']
df_clean['is_major_market'] = df_clean['primary_country'].isin(major_markets)

# English-speaking countries
english_countries = ['United States', 'United Kingdom', 'Canada', 'Australia', 'Ireland', 'New Zealand']
df_clean['is_english_speaking'] = df_clean['primary_country'].isin(english_countries)

# Asian markets
asian_countries = ['India', 'South Korea', 'Japan', 'China', 'Thailand', 'Philippines', 'Indonesia', 'Malaysia', 'Singapore']
df_clean['is_asian_market'] = df_clean['primary_country'].isin(asian_countries)

# European markets
european_countries = ['United Kingdom', 'France', 'Germany', 'Spain', 'Italy', 'Netherlands', 'Sweden', 'Norway', 'Denmark']
df_clean['is_european_market'] = df_clean['primary_country'].isin(european_countries)

print(f"  Created regional flags:")
print(f"    - Major markets: {df_clean['is_major_market'].sum():,} titles")
print(f"    - English-speaking: {df_clean['is_english_speaking'].sum():,} titles")
print(f"    - Asian markets: {df_clean['is_asian_market'].sum():,} titles")
print(f"    - European markets: {df_clean['is_european_market'].sum():,} titles")

# 4.4 Geographic Diversity Features
print(f"\n4.4 Geographic Diversity Metrics")


# Country popularity rank
country_popularity = df_clean['primary_country'].value_counts()
df_clean['country_popularity_rank'] = df_clean['primary_country'].map(
    {country: rank for rank, country in enumerate(country_popularity.index, 1)}
)

# Country production volume category
def classify_country_volume(rank):
    if rank <= 5:
        return 'Top_Producer'
    elif rank <= 15:
        return 'Major_Producer'
    elif rank <= 50:
        return 'Medium_Producer'
    else:
        return 'Small_Producer'

df_clean['country_volume_category'] = df_clean['country_popularity_rank'].apply(classify_country_volume)

print(f"Created country_popularity_rank (1 to {df_clean['country_popularity_rank'].max()})")
print(f"Created country_volume_category: {df_clean['country_volume_category'].value_counts().to_dict()}")

geographic_features = ['primary_country', 'continent', 'country_count', 'is_international', 
                      'is_major_market', 'is_english_speaking']
print(f"\nCreated {len(geographic_features)} geographic features successfully!")


4. GEOGRAPHIC FEATURE ENGINEERING

4.1 Country Data Processing
  Extracted primary_country (86 unique countries)
  Created country_count (range: 1 to 1)
  Created is_international flag (0 international productions)

4.2 Continent Classification
  Mapped countries to continents:
    - North America: 3,649 titles (41.5%)
    - Asia: 2,694 titles (30.7%)
    - Europe: 1,460 titles (16.6%)
    - Unknown: 336 titles (3.8%)
    - Africa: 281 titles (3.2%)
    - South America: 236 titles (2.7%)
    - Oceania: 131 titles (1.5%)

4.3 Regional Classification Features
  Created regional flags:
    - Major markets: 5,407 titles
    - English-speaking: 4,308 titles
    - Asian markets: 1,915 titles
    - European markets: 1,300 titles

4.4 Geographic Diversity Metrics
Created country_popularity_rank (1 to 86)
Created country_volume_category: {'Top_Producer': 5641, 'Major_Producer': 1731, 'Medium_Producer': 1335, 'Small_Producer': 80}

Created 6 geographic features successfully!


---
## 5. Content Feature Engineering 


In [9]:
print("5. CONTENT FEATURE ENGINEERING ")


# 5.1 Duration Processing
print(f"\n5.1 Duration Feature Processing")


if 'duration' in df_clean.columns:
    # Parse duration using utility function
    df_clean['duration_minutes'] = df_clean['duration'].apply(parse_duration)
    
    # Content type flag
    df_clean['is_movie'] = df_clean['type'] == 'Movie'
    
    # Duration categories for movies
    df_clean['is_short_content'] = (df_clean['is_movie']) & (df_clean['duration_minutes'] < 90)
    df_clean['is_long_content'] = (df_clean['is_movie']) & (df_clean['duration_minutes'] > 150)
    
    # Movie duration categories
    def classify_movie_duration(row):
        if row['type'] != 'Movie':
            return 'Not_Movie'
        elif row['duration_minutes'] < 60:
            return 'Short_Film'
        elif row['duration_minutes'] < 90:
            return 'Standard_Short'
        elif row['duration_minutes'] < 120:
            return 'Standard_Length'
        elif row['duration_minutes'] < 150:
            return 'Long_Film'
        else:
            return 'Epic_Length'
    
    df_clean['movie_duration_category'] = df_clean.apply(classify_movie_duration, axis=1)
    
    # TV show season analysis
    tv_shows = df_clean['type'] == 'TV Show'
    df_clean['tv_seasons'] = np.where(tv_shows, df_clean['duration_minutes'] / 10, np.nan)  # Convert back to seasons
    
    print(f"  Parsed duration_minutes (range: {df_clean['duration_minutes'].min():.0f} to {df_clean['duration_minutes'].max():.0f})")
    print(f"  Created movie duration categories: {df_clean['movie_duration_category'].value_counts().to_dict()}")
    print(f"  Created short/long content flags")

# 5.2 Genre Processing
print(f"\n5.2 Genre Feature Processing")

if 'listed_in' in df_clean.columns:
    # Use utility function to clean genres
    df_clean['genres_list'] = df_clean['listed_in'].apply(clean_genres)
    
    # Count number of genres
    df_clean['genre_count'] = df_clean['genres_list'].str.len()
    
    # Extract primary genre (first listed)
    df_clean['primary_genre'] = df_clean['genres_list'].str[0]
    
    # Genre diversity flag
    df_clean['is_multi_genre'] = df_clean['genre_count'] > 1
    
    # Popular genre flags
    top_genres = df_clean['primary_genre'].value_counts().head(10).index
    for genre in top_genres:
        genre_clean = genre.replace(' ', '_').replace('-', '_').lower()
        df_clean[f'is_{genre_clean}'] = df_clean['primary_genre'] == genre
    
    # Genre categories
    def classify_genre_type(genre):
        if pd.isna(genre):
            return 'Unknown'
        
        genre_lower = str(genre).lower()
        
        if any(word in genre_lower for word in ['action', 'thriller', 'crime']):
            return 'Action_Thriller'
        elif any(word in genre_lower for word in ['comedy', 'romantic']):
            return 'Comedy_Romance'
        elif any(word in genre_lower for word in ['drama', 'documentary']):
            return 'Drama_Documentary'
        elif any(word in genre_lower for word in ['horror', 'sci-fi', 'fantasy']):
            return 'Horror_SciFi'
        elif any(word in genre_lower for word in ['children', 'family', 'kids']):
            return 'Family_Kids'
        else:
            return 'Other'
    
    df_clean['genre_category'] = df_clean['primary_genre'].apply(classify_genre_type)
    
    print(f"  Processed {df_clean['listed_in'].nunique()} unique genre combinations")
    print(f"  Created genre_count (range: {df_clean['genre_count'].min():.0f} to {df_clean['genre_count'].max():.0f})")
    print(f"  Extracted primary_genre ({df_clean['primary_genre'].nunique()} unique)")
    print(f"  Created genre_category: {df_clean['genre_category'].value_counts().to_dict()}")

# 5.3 Rating Processing
print(f"\n5.3 Content Rating Processing")

if 'rating' in df_clean.columns:
    # Adult content classification
    adult_ratings = ['R', 'NC-17', 'TV-MA', 'NR']  # Added NR as potentially adult
    df_clean['is_adult_content'] = df_clean['rating'].isin(adult_ratings)
    
    # Family-friendly classification
    family_ratings = ['G', 'PG', 'TV-G', 'TV-Y', 'TV-Y7', 'TV-Y7-FV']
    df_clean['is_family_friendly'] = df_clean['rating'].isin(family_ratings)
    
    # Teen content
    teen_ratings = ['PG-13', 'TV-14']
    df_clean['is_teen_content'] = df_clean['rating'].isin(teen_ratings)
    
    # Rating strictness score (higher = more restrictive)
    rating_strictness = {
        'G': 1, 'TV-G': 1, 'TV-Y': 1,
        'PG': 2, 'TV-Y7': 2, 'TV-Y7-FV': 2,
        'PG-13': 3, 'TV-14': 3,
        'R': 4, 'TV-MA': 4,
        'NC-17': 5, 'NR': 3  # NR assumed moderate
    }
    df_clean['rating_strictness_score'] = df_clean['rating'].map(rating_strictness).fillna(3)
    
    # Rating category
    def classify_rating_category(rating):
        if rating in ['G', 'TV-G', 'TV-Y']:
            return 'All_Ages'
        elif rating in ['PG', 'TV-Y7', 'TV-Y7-FV']:
            return 'Family'
        elif rating in ['PG-13', 'TV-14']:
            return 'Teen'
        elif rating in ['R', 'TV-MA']:
            return 'Adult'
        elif rating in ['NC-17']:
            return 'Restricted'
        else:
            return 'Unrated'
    
    df_clean['rating_category'] = df_clean['rating'].apply(classify_rating_category)
    
    print(f"  Created content rating flags:")
    print(f"    - Adult content: {df_clean['is_adult_content'].sum():,} titles")
    print(f"    - Family-friendly: {df_clean['is_family_friendly'].sum():,} titles")
    print(f"    - Teen content: {df_clean['is_teen_content'].sum():,} titles")
    print(f"  Created rating_category: {df_clean['rating_category'].value_counts().to_dict()}")

# 5.4 Cast and Director Features
print(f"\n5.4 Cast and Director Features")

if 'director' in df_clean.columns:
    # Director count
    df_clean['director_count'] = df_clean['director'].str.split(',').str.len()
    df_clean['director_count'] = df_clean['director_count'].fillna(1)
    
    # Has known director
    df_clean['has_known_director'] = ~df_clean['director'].str.contains('Unknown', na=True)
    
    print(f"  Created director_count (range: {df_clean['director_count'].min():.0f} to {df_clean['director_count'].max():.0f})")
    print(f"  Has known director: {df_clean['has_known_director'].sum():,} titles")

if 'cast' in df_clean.columns:
    # Cast count
    df_clean['cast_count'] = df_clean['cast'].str.split(',').str.len()
    df_clean['cast_count'] = df_clean['cast_count'].fillna(0)
    
    # Has known cast
    df_clean['has_known_cast'] = ~df_clean['cast'].str.contains('Unknown', na=True)
    
    # Large cast flag
    df_clean['has_large_cast'] = df_clean['cast_count'] > 10
    
    print(f"  Created cast_count (range: {df_clean['cast_count'].min():.0f} to {df_clean['cast_count'].max():.0f})")
    print(f"  Has known cast: {df_clean['has_known_cast'].sum():,} titles")
    print(f"  Large cast (>10): {df_clean['has_large_cast'].sum():,} titles")

content_features = ['duration_minutes', 'is_movie', 'primary_genre', 'genre_count', 
                   'is_adult_content', 'is_family_friendly', 'rating_category']
print(f"\nCreated {len(content_features)} content features successfully!")


5. CONTENT FEATURE ENGINEERING 

5.1 Duration Feature Processing
  Parsed duration_minutes (range: 3 to 312)
  Created movie duration categories: {'Standard_Length': 3091, 'Not_Movie': 2663, 'Standard_Short': 1380, 'Long_Film': 934, 'Short_Film': 457, 'Epic_Length': 262}
  Created short/long content flags

5.2 Genre Feature Processing
  Processed 513 unique genre combinations
  Created genre_count (range: 1 to 3)
  Extracted primary_genre (36 unique)
  Created genre_category: {'Other': 4067, 'Drama_Documentary': 1665, 'Action_Thriller': 1362, 'Family_Kids': 990, 'Comedy_Romance': 403, 'Horror_SciFi': 300}

5.3 Content Rating Processing
  Created content rating flags:
    - Adult content: 4,085 titles
    - Family-friendly: 1,193 titles
    - Teen content: 2,645 titles
  Created rating_category: {'Adult': 4003, 'Teen': 2645, 'Unrated': 943, 'Family': 626, 'All_Ages': 567, 'Restricted': 3}

5.4 Cast and Director Features
  Created director_count (range: 1 to 13)
  Has known director: 8,7

---
## 6. Text Feature Engineering


In [10]:
print("6. TEXT FEATURE ENGINEERING ")


# 6.1 Check Existing Text Data
print(f"\n6.1 Text Data Assessment")

# Check what text columns exist
text_related_columns = [col for col in df_clean.columns if any(word in col.lower() for word in ['description', 'summary', 'plot', 'overview'])]
print(f"  • Found text-related columns: {text_related_columns if text_related_columns else 'None'}")

if text_related_columns:
    print(f"  • Using existing text data for analysis")
else:
    print(f"  • No existing text data found - will generate descriptions")

# 6.2 Create Description Column if Missing
print(f"\n6.2 Description Generation")

if 'description' not in df_clean.columns:
    print("  • Description column not found - generating descriptions from metadata")
    
    def generate_description(row):
        """Generate a description based on available metadata"""
        desc_parts = []
        
        # Start with content type and genre
        if pd.notna(row.get('type')) and pd.notna(row.get('listed_in')):
            content_type = str(row['type']).lower()
            genre = str(row['listed_in']).split(',')[0].strip()
            desc_parts.append(f"A {genre.lower()} {content_type}")
        
        # Add country information
        if pd.notna(row.get('country')):
            country = str(row['country']).split(',')[0].strip()
            if country != 'Unknown Country':
                desc_parts.append(f"from {country}")
        
        # Add release year
        if pd.notna(row.get('release_year')):
            year = int(row['release_year'])
            desc_parts.append(f"released in {year}")
        
        # Add rating information
        if pd.notna(row.get('rating')):
            rating = str(row['rating'])
            if rating in ['G', 'PG', 'TV-Y', 'TV-G']:
                desc_parts.append("suitable for all ages")
            elif rating in ['PG-13', 'TV-14']:
                desc_parts.append("recommended for teens and adults")
            elif rating in ['R', 'TV-MA', 'NC-17']:
                desc_parts.append("for mature audiences")
        
        # Add duration information
        if pd.notna(row.get('duration')):
            duration = str(row['duration'])
            if 'min' in duration:
                minutes = int(re.findall(r'\d+', duration)[0])
                if minutes < 90:
                    desc_parts.append("a short film")
                elif minutes > 150:
                    desc_parts.append("an epic length feature")
            elif 'Season' in duration:
                seasons = re.findall(r'\d+', duration)[0]
                desc_parts.append(f"spanning {seasons} season{'s' if int(seasons) > 1 else ''}")
        
        # Add director information if available
        if pd.notna(row.get('director')) and str(row['director']) != 'Unknown Director':
            director = str(row['director']).split(',')[0].strip()
            desc_parts.append(f"directed by {director}")
        
        # Add cast information if available
        if pd.notna(row.get('cast')) and str(row['cast']) != 'Unknown Cast':
            cast_list = str(row['cast']).split(',')
            if len(cast_list) >= 2:
                main_cast = ', '.join([cast.strip() for cast in cast_list[:2]])
                desc_parts.append(f"starring {main_cast}")
            elif len(cast_list) == 1:
                desc_parts.append(f"starring {cast_list[0].strip()}")
        
        # Create final description
        if desc_parts:
            description = desc_parts[0].capitalize()
            if len(desc_parts) > 1:
                description += " " + ", ".join(desc_parts[1:])
            description += "."
            
            # Add some variety based on genre
            genre_keywords = {
                'drama': ['compelling', 'emotional', 'powerful'],
                'comedy': ['hilarious', 'entertaining', 'funny'],
                'action': ['thrilling', 'exciting', 'action-packed'],
                'horror': ['scary', 'suspenseful', 'chilling'],
                'documentary': ['informative', 'educational', 'eye-opening'],
                'romance': ['romantic', 'heartwarming', 'touching'],
                'thriller': ['suspenseful', 'gripping', 'intense'],
                'sci-fi': ['futuristic', 'imaginative', 'innovative'],
                'fantasy': ['magical', 'enchanting', 'mystical']
            }
            
            # Add genre-specific adjective
            if pd.notna(row.get('listed_in')):
                genre_lower = str(row['listed_in']).lower()
                for genre_key, adjectives in genre_keywords.items():
                    if genre_key in genre_lower:
                        adj = np.random.choice(adjectives)
                        description = f"This {adj} " + description.lower()
                        break
            
            return description
        else:
            return f"A {str(row.get('type', 'content')).lower()} title on Netflix."
    
    # Generate descriptions for all rows
    df_clean['description'] = df_clean.apply(generate_description, axis=1)
    print(f"  ✓ Generated descriptions for {len(df_clean):,} titles")
    
    # Sample generated descriptions
    print(f"   Sample descriptions:")
    for i, desc in enumerate(df_clean['description'].head(3), 1):
        print(f"    {i}. {desc[:100]}...")

else:
    print("  • Description column already exists")

# Now create text features
df_clean['description_length'] = df_clean['description'].fillna('').astype(str).str.len()
df_clean['description_word_count'] = df_clean['description'].fillna('').astype(str).str.split().str.len()
df_clean['description_sentence_count'] = df_clean['description'].fillna('').astype(str).str.count(r'[.!?]+')

# Text quality flags
df_clean['has_description'] = df_clean['description_length'] > 10
df_clean['has_long_description'] = df_clean['description_length'] > 100

# Description categories
def classify_description_length(length):
    if length <= 10:
        return 'No_Description'
    elif length <= 50:
        return 'Short'
    elif length <= 150:
        return 'Medium'
    else:
        return 'Long'

df_clean['description_length_category'] = df_clean['description_length'].apply(classify_description_length)

text_features = ['description', 'description_length', 'description_word_count', 'has_description', 'description_length_category']

# 6.3 Text Feature Quality Summary
print(f"\n6.3 Text Feature Quality Summary")

print(f"   Description Statistics:")
print(f"    • Total descriptions: {len(df_clean):,}")
print(f"    • Average length: {df_clean['description_length'].mean():.0f} characters")
print(f"    • Average words: {df_clean['description_word_count'].mean():.0f} words")
print(f"    • Has meaningful description: {df_clean['has_description'].sum():,} ({df_clean['has_description'].mean()*100:.1f}%)")

print(f"\n   Description Length Categories:")
length_dist = df_clean['description_length_category'].value_counts()
for category, count in length_dist.items():
    print(f"    • {category}: {count:,} ({count/len(df_clean)*100:.1f}%)")

print(f"\n   Sample Generated Descriptions:")
sample_descriptions = df_clean['description'].sample(3, random_state=42)
for i, (idx, desc) in enumerate(sample_descriptions.items(), 1):
    title = df_clean.loc[idx, 'title'] if 'title' in df_clean.columns else f"Title {idx}"
    print(f"    {i}. {title}: {desc}")

print(f"\nCreated {len(text_features)} text features successfully!")
print(f"Description column ready for text mining analysis!")


6. TEXT FEATURE ENGINEERING 

6.1 Text Data Assessment
  • Found text-related columns: None
  • No existing text data found - will generate descriptions

6.2 Description Generation
  • Description column not found - generating descriptions from metadata
  ✓ Generated descriptions for 8,787 titles
   Sample descriptions:
    1. A documentaries movie from United States, released in 2020, recommended for teens and adults, direct...
    2. This action-packed a crime tv shows tv show from france, released in 2021, for mature audiences, spa...
    3. This compelling a tv dramas tv show from united states, released in 2021, for mature audiences, span...

6.3 Text Feature Quality Summary
   Description Statistics:
    • Total descriptions: 8,787
    • Average length: 123 characters
    • Average words: 20 words
    • Has meaningful description: 8,787 (100.0%)

   Description Length Categories:
    • Medium: 8,359 (95.1%)
    • Long: 428 (4.9%)

   Sample Generated Descriptions:
    1. Your Exc

---
## 7. Derived Analytics Features 


In [12]:
print("7. DERIVED ANALYTICS FEATURES ")


# 7.1 Content Popularity Proxies
print(f"\n7.1 Content Popularity Features")

# Genre popularity rank
if 'primary_genre' in df_clean.columns:
    genre_popularity = df_clean['primary_genre'].value_counts()
    df_clean['genre_popularity_rank'] = df_clean['primary_genre'].map(
        {genre: rank for rank, genre in enumerate(genre_popularity.index, 1)}
    )
    print(f"  Created genre_popularity_rank (1 to {df_clean['genre_popularity_rank'].max()})")

# Country-genre combination popularity
if 'primary_country' in df_clean.columns and 'primary_genre' in df_clean.columns:
    df_clean['country_genre_combo'] = df_clean['primary_country'] + "_" + df_clean['primary_genre']
    combo_popularity = df_clean['country_genre_combo'].value_counts()
    df_clean['combo_popularity_rank'] = df_clean['country_genre_combo'].map(
        {combo: rank for rank, combo in enumerate(combo_popularity.index, 1)}
    )
    print(f"  Created country-genre combination features")

# 7.2 Content Strategy Features
print(f"\n7.2 Content Strategy Features")

# Netflix originals proxy (recent content from major markets)
if all(col in df_clean.columns for col in ['is_recent_content', 'is_major_market']):
    df_clean['likely_netflix_original'] = df_clean['is_recent_content'] & df_clean['is_major_market']
    print(f"  Netflix originals proxy: {df_clean['likely_netflix_original'].sum():,} titles")

# Content diversity score per country
if 'primary_country' in df_clean.columns:
    country_diversity = df_clean.groupby('primary_country')['primary_genre'].nunique()
    df_clean['country_genre_diversity'] = df_clean['primary_country'].map(country_diversity)
    print(f"  Created country genre diversity scores")

analytics_features = ['genre_popularity_rank', 'likely_netflix_original', 'country_genre_diversity']
print(f"\nCreated {len(analytics_features)} analytics features successfully!")


7. DERIVED ANALYTICS FEATURES 

7.1 Content Popularity Features
  Created genre_popularity_rank (1 to 36)
  Created country-genre combination features

7.2 Content Strategy Features
  Netflix originals proxy: 3,175 titles
  Created country genre diversity scores

Created 3 analytics features successfully!


---
## 8. Data Quality Validation 


In [13]:
print("8. DATA QUALITY VALIDATION ")


# 8.1 Feature Validation
print(f"\n8.1 Feature Quality Validation")

validation_results = {}

# Check for negative durations
if 'duration_minutes' in df_clean.columns:
    negative_durations = (df_clean['duration_minutes'] < 0).sum()
    validation_results['negative_durations'] = negative_durations
    print(f"  • Negative durations: {negative_durations}")

# Check for future release years
if 'release_year' in df_clean.columns:
    future_releases = (df_clean['release_year'] > datetime.now().year).sum()
    validation_results['future_releases'] = future_releases
    print(f"  • Future release years: {future_releases}")

# Check for impossible content ages
if 'content_age_when_added' in df_clean.columns:
    impossible_ages = (df_clean['content_age_when_added'] < -5).sum()  # Allow some tolerance
    validation_results['impossible_ages'] = impossible_ages
    print(f"  • Impossible content ages: {impossible_ages}")

# 8.2 Data Completeness Check
print(f"\n8.2 Data Completeness Check")

key_features = ['title', 'type', 'primary_country', 'primary_genre', 'release_year', 'date_added_year']
completeness_report = {}

for feature in key_features:
    if feature in df_clean.columns:
        missing_count = df_clean[feature].isnull().sum()
        completeness_pct = ((len(df_clean) - missing_count) / len(df_clean)) * 100
        completeness_report[feature] = completeness_pct
        print(f"  • {feature}: {completeness_pct:.1f}% complete")

# 8.3 Statistical Validation
print(f"\n8.3 Statistical Validation")

numeric_features = df_clean.select_dtypes(include=[np.number]).columns
print(f"  • Numeric features: {len(numeric_features)}")
print(f"  • Total features created: {len(df_clean.columns)}")
print(f"  • Original features: {len(df_raw.columns)}")
print(f"  • New features added: {len(df_clean.columns) - len(df_raw.columns)}")

# Save validation report
validation_df = pd.DataFrame([validation_results]).T
validation_df.columns = ['Count']
validation_df.to_csv('../reports/data_quality/validation_report.csv')

completeness_df = pd.DataFrame([completeness_report]).T
completeness_df.columns = ['Completeness_Percentage']
completeness_df.to_csv('../reports/data_quality/completeness_report.csv')

print(f"\n Data quality validation completed!")


8. DATA QUALITY VALIDATION 

8.1 Feature Quality Validation
  • Negative durations: 0
  • Future release years: 0
  • Impossible content ages: 0

8.2 Data Completeness Check
  • title: 100.0% complete
  • type: 100.0% complete
  • primary_country: 100.0% complete
  • primary_genre: 100.0% complete
  • release_year: 100.0% complete
  • date_added_year: 100.0% complete

8.3 Statistical Validation
  • Numeric features: 23
  • Total features created: 77
  • Original features: 10
  • New features added: 67

 Data quality validation completed!


---
## 9. Final Export & Summary 


In [15]:
print("9. FINAL EXPORT & SUMMARY ")


# 9.1 Feature Summary
print(f"\n9.1 Feature Engineering Summary")

original_features = set(df_raw.columns)
new_features = [col for col in df_clean.columns if col not in original_features]

print(f" Dataset Transformation Summary:")
print(f"  • Original dataset: {df_raw.shape[0]:,} rows × {df_raw.shape[1]} columns")
print(f"  • Final dataset: {df_clean.shape[0]:,} rows × {df_clean.shape[1]} columns")
print(f"  • Data reduction: {df_raw.shape[0] - df_clean.shape[0]:,} rows removed")
print(f"  • New features created: {len(new_features)}")

print(f"\n New Features Created:")
feature_categories = {
    'Temporal': [f for f in new_features if any(x in f for x in ['date', 'year', 'age', 'recent', 'classic', 'era', 'decade'])],
    'Geographic': [f for f in new_features if any(x in f for x in ['country', 'continent', 'international', 'market'])],
    'Content': [f for f in new_features if any(x in f for x in ['duration', 'genre', 'rating', 'adult', 'family', 'movie'])],
    'Text': [f for f in new_features if any(x in f for x in ['description', 'text'])],
    'Analytics': [f for f in new_features if any(x in f for x in ['rank', 'score', 'popularity', 'diversity'])]
}

for category, features in feature_categories.items():
    if features:
        print(f"  {category} ({len(features)}): {', '.join(features[:3])}{'...' if len(features) > 3 else ''}")

# 9.2 Export Cleaned Dataset
print(f"\n9.2 Export Cleaned Dataset")

# Export to CSV
output_path = '../data/processed/netflix_cleaned.csv'
df_clean.to_csv(output_path, index=False)
file_size_mb = os.path.getsize(output_path) / (1024 * 1024)
print(f" Cleaned dataset exported to: {output_path}")
print(f" File size: {file_size_mb:.2f} MB")

# Export feature dictionary
feature_dict = {
    'original_features': list(original_features),
    'new_features': new_features,
    'feature_categories': feature_categories,
    'processing_date': datetime.now().isoformat()
}

import json
with open('../reports/data_quality/feature_dictionary.json', 'w') as f:
    json.dump(feature_dict, f, indent=2)
print(f" Feature dictionary saved to: reports/data_quality/feature_dictionary.json")

# Optional database export
try:
    engine = get_engine()
    if engine:
        df_clean.to_sql('netflix_cleaned', engine, if_exists='replace', index=False)
        print(f"Dataset also exported to PostgreSQL database")
except Exception as e:
    print(f"  Database export failed: {str(e)[:50]}...")


print(f" NETFLIX DATA CLEANING & FEATURE ENGINEERING COMPLETE!")

print(f" Processing Summary:")
print(f"   • Started with: {df_raw.shape[0]:,} records")
print(f"   • Final dataset: {df_clean.shape[0]:,} records ({len(df_clean.columns)} features)")
print(f"   • Created: {len(new_features)} new analytical features")
print(f"   • Data quality: Validated and exported")
print(f"   • Ready for: EDA, ML modeling, and dashboard creation")
print(f" Completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")



9. FINAL EXPORT & SUMMARY 

9.1 Feature Engineering Summary
 Dataset Transformation Summary:
  • Original dataset: 8,790 rows × 10 columns
  • Final dataset: 8,787 rows × 77 columns
  • Data reduction: 3 rows removed
  • New features created: 67

 New Features Created:
  Temporal (12): date_added_year, date_added_month, date_added_day...
  Geographic (13): primary_country, country_count, is_international...
  Content (17): duration_minutes, is_movie, movie_duration_category...
  Text (7): description, description_length, description_word_count...
  Analytics (6): content_freshness_score, country_popularity_rank, rating_strictness_score...

9.2 Export Cleaned Dataset
 Cleaned dataset exported to: ../data/processed/netflix_cleaned.csv
 File size: 5.80 MB
 Feature dictionary saved to: reports/data_quality/feature_dictionary.json
  Database export failed: (psycopg2.OperationalError) connection to server a...
 NETFLIX DATA CLEANING & FEATURE ENGINEERING COMPLETE!
 Processing Summary:
   • S

---